# 如何派发自定义回调事件

:::info 前提条件

本指南假定您熟悉以下概念：

- [回调](/docs/concepts/callbacks)
- [自定义回调处理器](/docs/how_to/custom_callbacks)
- [流事件API](/docs/concepts/streaming#streamevents)

:::

在某些情况下，您可能希望从 [Runnable](/docs/concepts/#runnable-interface) 内部派发一个自定义回调事件，以便在自定义回调处理器或通过 [流事件API](/docs/concepts/streaming#streamevents) 中显示该事件。

例如，如果您有一个具有多个步骤的长时间运行的工具，则可以在步骤之间派发自定义事件，并使用这些自定义事件来监控进度。
您还可以将这些自定义事件呈现给您的应用程序的最终用户，以向他们展示当前任务的进展情况。

要派发一个自定义事件，您需要为事件确定两个属性：`name` 和 `data`。

| 属性 | 类型 | 描述                                                                                              |
|-----------|------|----------------------------------------------------------------------------------------------------------|
| name      | string  | 事件的用户定义名称。                                                                       |
| data      | any     | 与事件相关联的数据。可以是任何内容，但我们建议将其设为可JSON序列化的数据。 |


:::info
- 自定义回调事件只能从现有的 `Runnable` 内部派发。
- 如果使用 `streamEvents`，您必须使用 `version: "v2"` 来消费自定义事件。
- 当前LangSmith还不支持发送或渲染自定义回调事件。
:::

## 流事件API

消费自定义事件最有用的方式是通过 [`.streamEvents()`](/docs/concepts/streaming#streamevents) 方法。

我们可以使用 `dispatchCustomEvent` API 从此方法中发出自定义事件。

```{=mdx}
:::caution 兼容性
派发自定义回调事件需要 `@langchain/core>=0.2.16`。有关升级 `@langchain/core` 时的注意事项，请参见 [此指南](/docs/how_to/installation/#installing-integration-packages)。

以下默认入口会触发 [`async_hooks`](https://nodejs.org/api/async_hooks.html) 的导入和初始化，以启用自动的 `RunnableConfig` 传递，但并非所有环境中都支持此功能。如果遇到导入问题，您必须从 `@langchain/core/callbacks/dispatch/web` 导入并手动传播 `RunnableConfig` 对象（参见以下示例）。
:::
```

In [3]:
import { RunnableLambda } from "@langchain/core/runnables";
import { dispatchCustomEvent } from "@langchain/core/callbacks/dispatch";

const reflect = RunnableLambda.from(async (value: string) => {
  await dispatchCustomEvent("event1", { reversed: value.split("").reverse().join("") });
  await dispatchCustomEvent("event2", 5);
  return value;
});

const eventStream = await reflect.streamEvents("hello world", { version: "v2" });

for await (const event of eventStream) {
  if (event.event === "on_custom_event") {
    console.log(event);
  }
}

{
  event: 'on_custom_event',
  run_id: '9eac217d-3a2d-4563-a91f-3bd49bee4b3d',
  name: 'event1',
  tags: [],
  metadata: {},
  data: { reversed: 'dlrow olleh' }
}
{
  event: 'on_custom_event',
  run_id: '9eac217d-3a2d-4563-a91f-3bd49bee4b3d',
  name: 'event2',
  tags: [],
  metadata: {},
  data: 5
}


如果你所处的网页环境不支持 `async_hooks`，则必须从网页入口导入并手动传播配置：

In [4]:
import { RunnableConfig, RunnableLambda } from "@langchain/core/runnables";
import { dispatchCustomEvent as dispatchCustomEventWeb } from "@langchain/core/callbacks/dispatch/web";

const reflect = RunnableLambda.from(async (value: string, config?: RunnableConfig) => {
  await dispatchCustomEventWeb("event1", { reversed: value.split("").reverse().join("") }, config);
  await dispatchCustomEventWeb("event2", 5, config);
  return value;
});

const eventStream = await reflect.streamEvents("hello world", { version: "v2" });

for await (const event of eventStream) {
  if (event.event === "on_custom_event") {
    console.log(event);
  }
}

{
  event: 'on_custom_event',
  run_id: 'dee1e4f0-c5ff-4118-9391-461a0dcc4cb2',
  name: 'event1',
  tags: [],
  metadata: {},
  data: { reversed: 'dlrow olleh' }
}
{
  event: 'on_custom_event',
  run_id: 'dee1e4f0-c5ff-4118-9391-461a0dcc4cb2',
  name: 'event2',
  tags: [],
  metadata: {},
  data: 5
}


## 回调处理程序

让我们看看如何使用 `dispatchCustomEvent` 发出自定义事件。

请记住，你**必须**在现有的 `Runnable` 内部调用 `dispatchCustomEvent`。

In [6]:
import { RunnableConfig, RunnableLambda } from "@langchain/core/runnables";
import { dispatchCustomEvent } from "@langchain/core/callbacks/dispatch";

const reflect = RunnableLambda.from(async (value: string) => {
  await dispatchCustomEvent("event1", { reversed: value.split("").reverse().join("") });
  await dispatchCustomEvent("event2", 5);
  return value;
});

await reflect.invoke("hello world", {
  callbacks: [{
    handleCustomEvent(eventName, data, runId) {
      console.log(eventName, data, runId);
    },
  }]
});

event1 { reversed: 'dlrow olleh' } 9c3770ac-c83d-4626-9643-b5fd80eb5431
event2 5 9c3770ac-c83d-4626-9643-b5fd80eb5431
hello world


## 相关内容

你现在已经了解了如何在链中触发自定义事件。

你可以查阅更详细的[流式事件指南](/docs/how_to/streaming/#using-stream-events)，了解更多解析和接收链中中间步骤的方法。